In [ ]:
import json
import re
import os
import time3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dsit = pd.read_csv('/content/drive/MyDrive/testingfornorm/ipc_Nav_Hyperlocal_IPC_DS_IT (1).csv')

<ipython-input-2-a184545d299d>:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  dsit = pd.read_csv('/content/drive/MyDrive/testingfornorm/ipc_Nav_Hyperlocal_IPC_DS_IT (1).csv')


In [ ]:
inv = pd.read_csv('/content/drive/MyDrive/testingfornorm/ipc_Nav_Hyperlocal_IPC_Inventory_Snapshot (1).csv')
dstarget = pd.read_csv('/content/drive/MyDrive/testingfornorm/ipc_Nav_Hyperlocal_IPC_DS_Target_Master.csv')
dsnetwork = pd.read_csv('/content/drive/MyDrive/testingfornorm/ipc_Nav_IPC_Hyperlocal_DS_Network_Master.csv')

<ipython-input-3-5b710cd9fc10>:2: DtypeWarning: Columns (11,29) have mixed types. Specify dtype option on import or set low_memory=False.
  dstarget = pd.read_csv('/content/drive/MyDrive/testingfornorm/ipc_Nav_Hyperlocal_IPC_DS_Target_Master.csv')


In [ ]:
import gc
gc.collect()

0

In [ ]:
dsnetwork.head(3)

,City,DS,DS_Name,SH_P0,SH_P1,NL_P0,NL_P1,L_P0,L_P1,SH_Reverse,NL_Reverse,L_Reverse,source
0,Ahmedabad,ahm_101_wh_hl_01,Chandkheda,bhi_pad_wh_nl_05nl,NaN,bhi_pad_wh_nl_01nl,bhi_vas_wh_nl_01nl,amd_bav_wh_l_03l,NaN,bhi_pad_wh_nl_05nl,bhi_pad_wh_nl_01nl,NaN,bhi_pad_wh_nl_05nl
1,Ahmedabad,ahm_102_wh_hl_01,Mahadev Nagar Tekra,bhi_pad_wh_nl_05nl,NaN,bhi_pad_wh_nl_01nl,bhi_vas_wh_nl_01nl,amd_bav_wh_l_03l,NaN,bhi_pad_wh_nl_05nl,bhi_pad_wh_nl_01nl,NaN,bhi_pad_wh_nl_05nl
2,Ahmedabad,ahm_103_wh_hl_01,Gota,bhi_pad_wh_nl_05nl,NaN,bhi_pad_wh_nl_01nl,bhi_vas_wh_nl_01nl,amd_bav_wh_l_03l,NaN,bhi_pad_wh_nl_05nl,bhi_pad_wh_nl_01nl,NaN,bhi_pad_wh_nl_05nl


In [ ]:
len(dsnetwork)

322

In [ ]:
dsnetwork['source'] = np.where(dsnetwork['SH_P1'].isnull(), dsnetwork['SH_P0'], dsnetwork['SH_P1'])

In [ ]:
dsnetwork = dsnetwork[['DS','source']]


In [ ]:
source_counts = dsnetwork['source'].value_counts()
source_counts

,count
source,
bhi_pad_wh_nl_05nl,87
ben_hos_wh_nl_02nl,72
new_new_wh_nl_01nl,61
bin_sh_wh_nl_01nl,41
ulu_sh_wh_nl_01nl,33


In [ ]:
dsnetwork.head(3)

,DS,source
0,ahm_101_wh_hl_01,bhi_pad_wh_nl_05nl
1,ahm_102_wh_hl_01,bhi_pad_wh_nl_05nl
2,ahm_103_wh_hl_01,bhi_pad_wh_nl_05nl


In [ ]:
inv = inv[inv['fc_area']=='store']

In [ ]:
inv = inv[['fsn','fc','atp']]

In [ ]:
inv.head(3)

,fsn,fc,atp
0,INSH8HZ3RPTZHJS7,ahm_103_wh_hl_01,2
1,VESG9NPDGAZFSSVQ,ahm_103_wh_hl_01,1
2,INSH8HZ3RPTZHJS7,ahm_109_wh_hl_01,1


In [ ]:
dstarget = dstarget[dstarget['Live'] == 1]

In [ ]:
dstarget = dstarget[['FSN', 'DS' , 'Target_Units']]

In [ ]:
dstarget = dstarget.rename(columns={'Target_Units': 'norm', 'DS': 'destination', 'FSN' : 'fsn'})

In [ ]:
dstarget.head(3)

,fsn,destination,norm
0,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1
1,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1
2,ABEG5C8HYGGCGP6H,ahm_103_wh_hl_01,1


In [ ]:
dsit = dsit[['fsn' , 'Dest_FC'	 , 'IWIT_Intransit']]

In [ ]:
dsit = dsit.rename(columns={'Dest_FC': 'destination', 'IWIT_Intransit': 'Intransit_Qty'})

In [ ]:
from datetime import datetime
dsit = dsit[dsit['Inwarding_Status'].isin(['IN_TRANSIT', 'INWARDING', 'INITIATED'])]
dsit['Creation_Date'] = pd.to_datetime(dsit['Creation_Date'], format="%Y-%m-%d %H:%M:%S")
dsit['days'] = (pd.Timestamp.now() - dsit['Creation_Date']).dt.total_seconds() / (3600 * 24)
dsit = dsit[dsit['days'] < 7]

In [ ]:
dsit = dsit.reset_index(drop=True)

In [ ]:
dsit.head(3)

,fsn,destination,Intransit_Qty
0,TPSFHUSMJDNQR3MA,ahm_101_wh_hl_01,1
1,DEOGDQ9VYHQEKFUN,ahm_101_wh_hl_01,1
2,DAJEUARY8GKAKXUZ,ahm_101_wh_hl_01,1


In [ ]:
len(dsit)

704229

In [ ]:
len(dstarget)

3169079

In [ ]:
merged_df = pd.merge(dstarget, dsit, on=['fsn', 'destination'], how='left')

In [ ]:
merged_df.head(3)

,fsn,destination,norm,Intransit_Qty
0,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1,NaN
1,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1,NaN
2,ABEG5C8HYGGCGP6H,ahm_103_wh_hl_01,1,NaN


In [ ]:
inv.head(3)

,fsn,fc,atp
0,INSH8HZ3RPTZHJS7,ahm_103_wh_hl_01,2
1,VESG9NPDGAZFSSVQ,ahm_103_wh_hl_01,1
2,INSH8HZ3RPTZHJS7,ahm_109_wh_hl_01,1


In [ ]:
merged_df_copy = merged_df.copy()

In [ ]:
merged_df_copy= merged_df_copy.rename(columns ={'destination': 'fc' })

In [ ]:
exclude_fcs = ['bhi_pad_wh_nl_05nl', 'ben_hos_wh_nl_02nl', 'new_new_wh_nl_01nl', 'bin_sh_wh_nl_01nl', 'ulu_sh_wh_nl_01nl']
filtered_inv = inv[~inv['fc'].isin(exclude_fcs)]
final_df = pd.merge(merged_df_copy, filtered_inv, on=['fsn', 'fc'], how='left')


In [ ]:
final_df = final_df.rename(columns={'atp': 'ds_atp', 'fc' : 'destination'})
final_df.head()


,fsn,destination,norm,Intransit_Qty,ds_atp
0,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1,NaN,1.0
1,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1,NaN,2.0
2,ABEG5C8HYGGCGP6H,ahm_103_wh_hl_01,1,NaN,1.0
3,ABEG5C8HYGGCGP6H,ahm_116_wh_hl_01,1,NaN,1.0
4,ABEG5C8HYGGCGP6H,ahm_112_wh_hl_01,1,NaN,1.0


In [ ]:
filtered_inv_1 = inv[inv['fc'].isin(exclude_fcs)]

In [ ]:
filtered_inv_1 = filtered_inv_1.reset_index(drop=True)


In [ ]:
filtered_inv_1 = filtered_inv_1.rename(columns={'fc': 'source'})

In [ ]:
filtered_inv_1.head(3)


,fsn,source,atp
0,JWSH46JYAYVF3PVH,ben_hos_wh_nl_02nl,6
1,HTTG8C5DGGMBGMTA,new_new_wh_nl_01nl,0
2,SMAHFRSJWZS6HQWY,ben_hos_wh_nl_02nl,358


In [ ]:
dsnetwork.head(3)

,DS,source
0,ahm_101_wh_hl_01,bhi_pad_wh_nl_05nl
1,ahm_102_wh_hl_01,bhi_pad_wh_nl_05nl
2,ahm_103_wh_hl_01,bhi_pad_wh_nl_05nl


In [ ]:
final_df.head(3)

,fsn,destination,norm,Intransit_Qty,ds_atp
0,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1,NaN,1.0
1,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1,NaN,2.0
2,ABEG5C8HYGGCGP6H,ahm_103_wh_hl_01,1,NaN,1.0


In [ ]:
final_df_1 = pd.merge(final_df, dsnetwork, left_on='destination', right_on='DS', how='left')
# final_df_1 = final_df_1.rename(columns={'source': 'source_ds'})

,fsn,destination,norm,Intransit_Qty,ds_atp,DS,source
0,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1,NaN,1.0,ahm_101_wh_hl_01,bhi_pad_wh_nl_05nl
1,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1,NaN,2.0,ahm_102_wh_hl_01,bhi_pad_wh_nl_05nl
2,ABEG5C8HYGGCGP6H,ahm_103_wh_hl_01,1,NaN,1.0,ahm_103_wh_hl_01,bhi_pad_wh_nl_05nl
3,ABEG5C8HYGGCGP6H,ahm_116_wh_hl_01,1,NaN,1.0,ahm_116_wh_hl_01,bhi_pad_wh_nl_05nl
4,ABEG5C8HYGGCGP6H,ahm_112_wh_hl_01,1,NaN,1.0,ahm_112_wh_hl_01,bhi_pad_wh_nl_05nl


In [ ]:
final_df_1 = final_df_1.drop('DS', axis=1)

In [ ]:
final_df_1.head()

,fsn,destination,norm,Intransit_Qty,ds_atp,source
0,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1,NaN,1.0,bhi_pad_wh_nl_05nl
1,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1,NaN,2.0,bhi_pad_wh_nl_05nl
2,ABEG5C8HYGGCGP6H,ahm_103_wh_hl_01,1,NaN,1.0,bhi_pad_wh_nl_05nl
3,ABEG5C8HYGGCGP6H,ahm_116_wh_hl_01,1,NaN,1.0,bhi_pad_wh_nl_05nl
4,ABEG5C8HYGGCGP6H,ahm_112_wh_hl_01,1,NaN,1.0,bhi_pad_wh_nl_05nl


In [ ]:
final_df_2 = pd.merge(final_df_1, filtered_inv_1, on=['fsn', 'source'], how='left')
final_df_2.head()


,fsn,destination,norm,Intransit_Qty,ds_atp,DS,source,atp
0,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1,NaN,1.0,ahm_101_wh_hl_01,bhi_pad_wh_nl_05nl,29.0
1,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1,NaN,1.0,ahm_101_wh_hl_01,bhi_pad_wh_nl_05nl,1.0
2,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1,NaN,2.0,ahm_102_wh_hl_01,bhi_pad_wh_nl_05nl,29.0
3,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1,NaN,2.0,ahm_102_wh_hl_01,bhi_pad_wh_nl_05nl,1.0
4,ABEG5C8HYGGCGP6H,ahm_103_wh_hl_01,1,NaN,1.0,ahm_103_wh_hl_01,bhi_pad_wh_nl_05nl,29.0


In [ ]:

def count_unique_destinations(df):
    unique_dest_counts = df.groupby(['fsn', 'source'])['destination'].nunique().reset_index()
    unique_dest_counts = unique_dest_counts.rename(columns={'destination': 'unique_destination_count'})
    return unique_dest_counts
unique_destination_counts_df = count_unique_destinations(final_df_2)


In [ ]:
unique_destination_counts_df.head()

,fsn,source,unique_destination_count
0,9780008333782,ben_hos_wh_nl_02nl,22
1,9780008333782,bhi_pad_wh_nl_05nl,21
2,9780008333782,bin_sh_wh_nl_01nl,25
3,9780008333782,new_new_wh_nl_01nl,34
4,9780008333782,ulu_sh_wh_nl_01nl,5


In [ ]:
import pandas as pd

merged_df_2 = final_df_2.merge(
    unique_destination_counts_df[['fsn', 'source', 'unique_destination_count']],
    on=['fsn', 'source'],
    how='left'
)


merged_df_2['atp'] = merged_df_2['atp'] / merged_df_2['unique_destination_count']




# final_df_2_updated = merged_df


(merged_df_2.head())


,fsn,destination,norm,Intransit_Qty,ds_atp,DS,source,atp,unique_destination_count
0,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1,NaN,1.0,ahm_101_wh_hl_01,bhi_pad_wh_nl_05nl,0.517857,56
1,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1,NaN,1.0,ahm_101_wh_hl_01,bhi_pad_wh_nl_05nl,0.017857,56
2,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1,NaN,2.0,ahm_102_wh_hl_01,bhi_pad_wh_nl_05nl,0.517857,56
3,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1,NaN,2.0,ahm_102_wh_hl_01,bhi_pad_wh_nl_05nl,0.017857,56
4,ABEG5C8HYGGCGP6H,ahm_103_wh_hl_01,1,NaN,1.0,ahm_103_wh_hl_01,bhi_pad_wh_nl_05nl,0.517857,56


In [ ]:
merged_df_2 = merged_df_2.drop(['unique_destination_count', 'DS'], axis=1)
merged_df_2 = merged_df_2.rename(columns={'atp': 'sh_pivot_atp'})
merged_df_2.head()


,fsn,destination,norm,Intransit_Qty,ds_atp,source,sh_pivot_atp
0,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1,NaN,1.0,bhi_pad_wh_nl_05nl,0.517857
1,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1,NaN,1.0,bhi_pad_wh_nl_05nl,0.017857
2,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1,NaN,2.0,bhi_pad_wh_nl_05nl,0.517857
3,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1,NaN,2.0,bhi_pad_wh_nl_05nl,0.017857
4,ABEG5C8HYGGCGP6H,ahm_103_wh_hl_01,1,NaN,1.0,bhi_pad_wh_nl_05nl,0.517857


In [ ]:
merged_df_2['Intransit_Qty'] = merged_df_2['Intransit_Qty'].fillna(0)
merged_df_2['transfer_quantity'] = merged_df_2.apply(lambda row: min(row['sh_pivot_atp'], max(0, row['norm'] - row['Intransit_Qty'] - row['ds_atp'])), axis=1)


In [ ]:
merged_df_2.head()

,fsn,destination,norm,Intransit_Qty,ds_atp,source,sh_pivot_atp,transfer_quantity
0,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1,0.0,1.0,bhi_pad_wh_nl_05nl,0.517857,0.0
1,ABEG5C8HYGGCGP6H,ahm_101_wh_hl_01,1,0.0,1.0,bhi_pad_wh_nl_05nl,0.017857,0.0
2,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1,0.0,2.0,bhi_pad_wh_nl_05nl,0.517857,0.0
3,ABEG5C8HYGGCGP6H,ahm_102_wh_hl_01,1,0.0,2.0,bhi_pad_wh_nl_05nl,0.017857,0.0
4,ABEG5C8HYGGCGP6H,ahm_103_wh_hl_01,1,0.0,1.0,bhi_pad_wh_nl_05nl,0.517857,0.0
